# 构建一个代理

语言模型本身无法采取行动 - 它们只输出文本。
LangChain 的一个重要用例是创建 **代理**。
[代理](/docs/concepts/agents) 是使用 [LLMs](/docs/concepts/chat_models) 作为推理引擎来确定采取哪些行动以及执行行动所需输入的系统。
执行操作后，可以将结果反馈到 LLM 中以确定是否需要更多操作，或者是否可以完成。这通常通过 [工具调用](/docs/concepts/tool_calling) 来实现。

在本教程中，我们将构建一个可以与搜索引擎交互的代理。您将能够向此代理提问，观察它调用搜索工具，并与之对话。

## 端到端代理

下面的代码片段表示一个完全功能的代理，它使用 LLM 来决定使用哪些工具。它配备了一个通用搜索工具。它具有对话记忆功能 - 这意味着它可以用作多轮聊天机器人。

在本指南的其余部分，我们将逐步介绍各个组件及其作用 - 但如果您只想获取一些代码并开始使用，请随意使用它！

In [ ]:
# 导入相关功能
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# 创建代理
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# 使用代理
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

Hello Bob! Since you didn't ask a specific question, I don't need to use any tools right now. I'm an AI assistant created by Anthropic to be helpful, honest, and harmless. Feel free to ask me anything and I'll do my best to provide a useful response or look up information using my capabilities.


In [3]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================

[{'text': 'To get the current weather for your location in San Francisco, I can use the tavily_search_results_json tool:', 'type': 'text'}, {'id': 'toolu_01AKa2MErG1CU3zRiGsvpBud', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01AKa2MErG1CU3zRiGsvpBud)
 Call ID: toolu_01AKa2MErG1CU3zRiGsvpBud
  Args:
    query: san francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739994486

## 设置

### Jupyter Notebook

本指南（以及文档中的大多数其他指南）使用 [Jupyter notebooks](https://jupyter.org/) 并假设读者也是如此。Jupyter notebooks 是学习如何使用 LLM 系统的完美交互环境，因为有时事情可能会出错（意外输出、API 停止等），观察这些情况是更好地理解使用 LLM 构建的好方法。

本教程和其他教程可能最方便在 Jupyter notebook 中运行。请参阅 [此处](https://jupyter.org/install) 了解安装说明。

### 安装

要安装 LangChain，请运行：

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

有关更多详细信息，请参阅我们的 [安装指南](/docs/how_to/installation)。

### LangSmith

您使用 LangChain 构建的许多应用程序将包含多个步骤和多次 LLM 调用。
随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么变得至关重要。
最好的方法是使用 [LangSmith](https://smith.langchain.com)。

在上面的链接注册后，请确保设置环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者，如果在 notebook 中，您可以使用以下方式设置：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Tavily

我们将使用 [Tavily](/docs/integrations/tools/tavily_search)（一个搜索引擎）作为工具。
为了使用它，您需要获取并设置一个 API 密钥：

```bash
export TAVILY_API_KEY="..."
```

或者，如果在 notebook 中，您可以使用以下方式设置：

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## 定义工具

我们首先需要创建我们想要使用的工具。我们的主要工具选择是 [Tavily](/docs/integrations/tools/tavily_search) - 一个搜索引擎。我们在 LangChain 中有一个内置工具，可以轻松使用 Tavily 搜索引擎作为工具。


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# 如果我们愿意，我们可以创建其他工具。
# 一旦我们拥有了所有想要的工具，我们可以将它们放入一个稍后引用的列表中。
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739993250, 'localtime': '2025-02-19 11:27'}, 'current': {'last_updated_epoch': 1739992500, 'last_updated': '2025-02-19 11:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 195, 'wind_dir': 'SSW', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 100, 'feelslike_c': 12.7, 'feelslike_f': 54.8, 'windchill_c': 9.1, 'windchill_f': 48.4, 'heatindex_c': 10.2, 'heatindex_f': 50.3, 'dewpoint_c': 9.8, 'dewpoint_f': 49.7, 'vis_km': 4.0, 'vis_miles': 2.0, 'uv': 1.4, 'gust_mph': 8.9, 'gust_kph': 14.4}}"}, {'url': 'https://weathershogun.com/weather/usa/ca/

## 使用语言模型

接下来，让我们学习如何使用语言模型调用工具。LangChain 支持许多不同的语言模型，您可以互换使用 - 选择您想要使用的模型！

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />


In [3]:
# | output: false
# | echo: false

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-sonnet-20240229")

您可以通过传入消息列表来调用语言模型。默认情况下，响应是一个 `content` 字符串。

In [4]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there!'

我们现在可以看到启用此模型进行工具调用是什么样子。为了启用它，我们使用 `.bind_tools` 将这些工具的知识赋予语言模型。

In [5]:
model_with_tools = model.bind_tools(tools)

我们现在可以调用模型。让我们首先用普通消息调用它，看看它如何响应。我们可以查看 `content` 字段以及 `tool_calls` 字段。

In [6]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello!
ToolCalls: []


现在，让我们尝试用一些输入调用它，这些输入会期望调用工具。

In [7]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather san francisco'}, 'id': 'toolu_01VTP7DUvSfgtYxsq9x4EwMp'}]


我们可以看到现在没有文本内容，但有一个工具调用！它希望我们调用 Tavily 搜索工具。

这还没有调用该工具 - 它只是告诉我们要调用它。为了实际调用它，我们需要创建我们的代理。

## 创建代理

现在我们已经定义了工具和 LLM，我们可以创建代理。我们将使用 [LangGraph](/docs/concepts/architecture/#langgraph) 来构建代理。
目前，我们正在使用高级接口来构建代理，但 LangGraph 的优点是这个高级接口由低级、高度可控的 API 支持，以防您想要修改代理逻辑。


现在，我们可以使用 LLM 和工具初始化代理。

请注意，我们传入的是 `model`，而不是 `model_with_tools`。这是因为 `create_react_agent` 会在底层为我们调用 `.bind_tools`。

In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## 运行代理

我们现在可以运行代理并进行一些查询！请注意，目前这些都是 **无状态** 查询（它不会记住以前的交互）。请注意，代理将在交互结束时返回 **最终** 状态（包括任何输入，我们稍后将看到如何仅获取输出）。

首先，让我们看看当不需要调用工具时它如何响应：

In [10]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='a820fcc5-9b87-457a-9af0-f21768143ee3'),
 AIMessage(content='Hello!', response_metadata={'id': 'msg_01VbC493X1VEDyusgttiEr1z', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 264, 'output_tokens': 5}}, id='run-0e0ddae8-a85b-4bd6-947c-c36c857a4698-0', usage_metadata={'input_tokens': 264, 'output_tokens': 5, 'total_tokens': 269})]

为了确切了解底层发生了什么（并确保它没有调用工具），我们可以查看 [LangSmith 跟踪](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)。

现在让我们尝试一个应该调用工具的示例。

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='1d6c96bb-4ddb-415c-a579-a07d5264de0d'),
 AIMessage(content=[{'id': 'toolu_01Y5EK4bw2LqsQXeaUv8iueF', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_0132wQUcEduJ8UKVVVqwJzM4', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 269, 'output_tokens': 61}}, id='run-26d5e5e8-d4fd-46d2-a197-87b95b10e823-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': 'toolu_01Y5EK4bw2LqsQXeaUv8iueF'}], usage_metadata={'input_tokens': 269, 'output_tokens': 61, 'total_tokens': 330}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localti

我们可以查看 [LangSmith 跟踪](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) 以确保它有效地调用了搜索工具。

## 流式消息

我们已经看到代理可以通过 `.invoke` 调用以获得最终响应。如果代理执行多个步骤，这可能需要一段时间。为了显示中间进度，我们可以在消息发生时流式返回消息。

In [14]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'text': 'Okay, let me look up the current weather for San Francisco using a search engine:', 'type': 'text'}, {'id': 'toolu_01H1brh5EZpZqtqHBxkosPtN', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01H1brh5EZpZqtqHBxkosPtN)
 Call ID: toolu_01H1brh5EZpZqtqHBxkosPtN
  Args:
    query: san francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739994486, 'localtime': '2025-02-19 11:48'},

## 流式令牌

除了流式返回消息外，流式返回令牌也很有用。
我们可以通过指定 `stream_mode="messages"` 来实现。


::: 注意

下面我们使用 `message.text()`，这需要 `langchain-core>=0.3.37`。

:::

In [21]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")



Base|d on the weather| search| results, here| are the key details| about the weather in| San Francisco:|

- The current temperature| in| San Francisco is aroun|d 55|-|56|°F (13|°|C).| Light| rain is occurring with| |100|% clou|d cover. |

-| Winds| are aroun|d 5-9| mph from| the south|-southwest.|

- The| forecast| for| the rest| of February| 2025 |shows da|ytime highs mostly| in the upper| 50s to| low| 60s°|F,| with overnight lows| in| the upper| 40s to| low| 50s°|F.|

-| Overall|, typical| cool| an|d show|ery late| winter weather is| expected in San Francisco| for the remainder| of February,| with a| mix| of rain| and dry| periods|.| Temperatures will be| season|able| for| this| time of year.|

So| in summary, San| Francisco is| experiencing light| rain an|d cool| temperatures currently, but| the late| winter forecast| shows typical mil|d and show|ery conditions| pers|isting through the en|d of the| month.| Let| me know if you| need any other| details about| the weather in the| cit

## 添加记忆

如前所述，此代理是无状态的。这意味着它不会记住以前的交互。为了给它记忆，我们需要传入一个检查点器。当传入检查点器时，我们还必须在调用代理时传入一个 `thread_id`（以便它知道要从哪个线程/对话中恢复）。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [12]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you again.", response_metadata={'id': 'msg_013C1z2ZySagEFwmU1EsysR2', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1162, 'output_tokens': 14}}, id='run-f878acfd-d195-44e8-9166-e2796317e3f8-0', usage_metadata={'input_tokens': 1162, 'output_tokens': 14, 'total_tokens': 1176})]}}
----


In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You mentioned your name is Bob when you introduced yourself earlier. So your name is Bob.', response_metadata={'id': 'msg_01WNwnRNGwGDRw6vRdivt6i1', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1184, 'output_tokens': 21}}, id='run-f5c0b957-8878-405a-9d4b-a7cd38efe81f-0', usage_metadata={'input_tokens': 1184, 'output_tokens': 21, 'total_tokens': 1205})]}}
----


示例 [LangSmith 跟踪](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

如果您想开始新的对话，您只需更改使用的 `thread_id`

In [15]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm afraid I don't actually know your name. As an AI assistant without personal information about you, I don't have a specific name associated with our conversation.", response_metadata={'id': 'msg_01NoaXNNYZKSoBncPcLkdcbo', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 267, 'output_tokens': 36}}, id='run-c9f7df3d-525a-4d8f-bbcf-a5b4a5d2e4b0-0', usage_metadata={'input_tokens': 267, 'output_tokens': 36, 'total_tokens': 303})]}}
----


## 结论

到此为止！在这个快速入门中，我们介绍了如何创建一个简单的代理。
然后我们展示了如何流式返回响应 - 不仅包括中间步骤，还包括令牌！
我们还添加了记忆功能，以便您可以与它进行对话。
代理是一个复杂的主题，有很多需要学习的内容！

有关代理的更多信息，请查看 [LangGraph](/docs/concepts/architecture/#langgraph) 文档。这有自己的一套概念、教程和操作指南。